*C++ presentation of the 'Templet application' commom interface*

In [2]:
#include <string>

struct SESSION{};
struct OWNER_SESSION:SESSION{};
struct USER_SESSION:SESSION{};

struct TOKEN{};
struct OWNER_TOKEN:TOKEN{};
struct USER_TOKEN:TOKEN{};

bool open_owner_session(/*in*/OWNER_TOKEN,/*out*/OWNER_SESSION&);
bool open_user_session(/*in*/USER_TOKEN,/*out*/USER_SESSION&);

bool close_owner_session(/*in*/OWNER_SESSION);
bool close_user_session(/*in*/USER_SESSION);

typedef int ORD;
typedef int TAG;
typedef int ID;

struct  DATA{};

typedef std::string NAME;

struct EVENT{
    ORD  ord;
    TAG  tag;
    DATA data;
    bool writer_is_owner;
    NAME writer_name;
};

bool write_event(/*in*/OWNER_SESSION,/*in*/TAG,/*in*/DATA,/*out*/ORD&);
bool read_event(/*in*/OWNER_SESSION, /*in*/ORD,/*out*/EVENT&);
bool reply_on_query(/*in*/OWNER_SESSION,/*in*/ORD,/*in*/DATA);

bool write_query(/*in*/USER_SESSION,/*in*/TAG,/*in*/DATA,/*out*/ORD&);
bool read_answer(/*in*/USER_SESSION,/*in*/ORD,/*out*/DATA&);

struct BLOB{
    ID  id;
    DATA data;
    NAME data_name;  
    bool writer_is_owner;
    NAME writer_name;
};

struct ANSWER{
    ORD  query_ord;
    NAME query_writer_name;
    DATA data;
};

bool upload_blob(/*in*/SESSION,/*in*/DATA,/*in*/NAME,/*out*/ID&);
bool download_blob(/*in*/SESSION,/*in*/ID,/*out*/BLOB&);
bool delete_blob(/*in*/OWNER_SESSION,/*in*/ID);

*C++ presentation of the 'Templet application' system interface*

In [ ]:
//???